In [1]:
import numpy as np
import pandas as pd
import re
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from underthesea import word_tokenize

In [2]:
# Load stopwords
STOP_WORD_FILE = 'vietnamese-stopwords.txt'
with open(STOP_WORD_FILE, 'r', encoding='utf-8') as file:
    stop_words = file.read().split('\n')

In [3]:
# Đọc dữ liệu
df = pd.read_csv("Products_ThoiTrangNam_raw.csv")

In [4]:
# Lọc các sản phẩm có giá bằng 0
zero_price_products = df[df['price'] == 0]

# Đếm số lượng sản phẩm theo từng 'sub_category'
sub_category_counts = zero_price_products['sub_category'].value_counts()

print("Số lượng sản phẩm có giá 0 theo từng danh mục con:")
print(sub_category_counts)

Số lượng sản phẩm có giá 0 theo từng danh mục con:
Kính Mắt Nam               503
Cà vạt & Nơ cổ             289
Khác                       103
Trang Phục Truyền Thống     30
Vớ/Tất                      14
Đồ Hóa Trang                12
Đồ Bộ                        5
Áo Khoác                     4
Quần Dài/Quần Âu             2
Áo                           1
Name: sub_category, dtype: int64


In [5]:
# Tính giá trung bình của từng sub_category và bỏ qua giá trị 0
mean_prices = df[df['price'] > 0].groupby('sub_category')['price'].mean()

# Thay thế giá 0 bằng giá trung bình của sub_category tương ứng
df.loc[df['price'] == 0, 'price'] = df['sub_category'].map(mean_prices)

# Kiểm tra lại xem còn sản phẩm nào có giá bằng 0 không
print("Số lượng sản phẩm còn giá 0:", (df['price'] == 0).sum())

Số lượng sản phẩm còn giá 0: 0


In [6]:
 #Bộ ký tự tiếng Việt
vietnamese_chars = (
    "a-zA-Z0-9_"
    "àáạảãâầấậẩẫăằắặẳẵ"
    "èéẹẻẽêềếệểễ"
    "ìíịỉĩ"
    "òóọỏõôồốộổỗơờớợởỡ"
    "ùúụủũưừứựửữ"
    "ỳýỵỷỹ"
    "đ"
    "ÀÁẠẢÃÂẦẤẬẨẪĂẰẮẶẲẴ"
    "ÈÉẸẺẼÊỀẾỆỂỄ"
    "ÌÍỊỈĨ"
    "ÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠ"
    "ÙÚỤỦŨƯỪỨỰỬỮ"
    "ỲÝỴỶỸ"
    "Đ"
)

In [7]:
pattern = f'^[{vietnamese_chars}]+$'

In [8]:
def is_valid_vietnamese(word):
    return re.match(pattern, word) is not None

In [9]:
def filter_vietnamese_words(text):
    if not isinstance(text, str):
        return ''
    words = text.split()
    clean_words = [w for w in words if is_valid_vietnamese(w)]
    return ' '.join(clean_words)

In [10]:
# Làm sạch dữ liệu
df['description_clean'] = df['description'].apply(filter_vietnamese_words)
df['Content'] = df['product_name'] + ' ' + df['description_clean'].apply(lambda x: ' '.join(x.split()[:100]))
df['Content_wt'] = df['Content'].apply(lambda x: word_tokenize(x, format="text"))

In [11]:
# TF-IDF
vectorizer = TfidfVectorizer(analyzer='word', stop_words=stop_words)
tfidf_matrix = vectorizer.fit_transform(df['Content_wt'])


In [12]:
# Cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [13]:
#  Tạo từ điển gợi ý TOP 9
top_k = 9
recommendations_dict = {}

In [14]:
for idx in range(len(df)):
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_k+1]
    product_id = df.loc[idx, 'product_id']
    similar_ids = [df.loc[i[0], 'product_id'] for i in sim_scores]
    recommendations_dict[product_id] = similar_ids

In [15]:
#  Lưu từ điển gợi ý (chỉ vài MB)
with open('recommendations_dict.pkl', 'wb') as f:
    pickle.dump(recommendations_dict, f)

In [16]:
#  Lưu lại dataframe nếu cần
with open('products_df.pkl', 'wb') as f:
    pickle.dump(df, f)